## From corpus to noun term frequency sparse matrix

In [1]:
from collections import defaultdict
from pprint import pprint
from konlpy.tag import Twitter

In [2]:
twitter = Twitter()

In [3]:
twitter.nouns("이게 바로 테스트라는 녀석")

['이', '바로', '테스트', '녀석']

In [4]:
preprocessed_data_fname = './data/small_naver_news/processed/corpus.txt'
model_folder = './tmp/'

In [5]:
with open(preprocessed_data_fname, encoding="utf-8") as f:
    docs = [doc.strip() for doc in f]

In [6]:
docs[:2]

["(의왕 국회사진기자단=연합뉴스) 김성태 '최순실 국조특위' 위원장이 26일 오전 경기도 의왕시 서울구치소에서 열린 현장청문회에 입장하고 있다. 2016.12.26  scoop@yna.co.kr",
 "(서울=연합뉴스) 이재희 기자 = 소녀시대의 태연이 26일 오후 서울 강남구 코엑스에서 열린 'SBS 어워즈 페스티벌(SAF) 가요대전'에서 포즈를 취하고 있다. 2016.12.26  scape@yna.co.kr"]

In [7]:
from collections import Counter

In [8]:
%%time
noun_counter = Counter([noun for doc in docs for noun in twitter.nouns(doc)])
print(len(noun_counter))

12612
CPU times: user 16.2 s, sys: 314 ms, total: 16.5 s
Wall time: 12.5 s


In [9]:
noun_counter

Counter({'연설': 16,
         '결선투표제': 25,
         '황실': 3,
         '전학': 1,
         '장': 54,
         '팟캐스트': 3,
         '몸속': 1,
         '정유년': 5,
         '성분': 19,
         '상임': 10,
         '보배': 3,
         '언론': 65,
         '북한': 85,
         '흑연': 1,
         '계명산': 1,
         '소회': 3,
         '양해각서': 3,
         '무안': 15,
         '달리': 19,
         '음': 20,
         '길주로': 1,
         '최수현': 1,
         '무궁무진': 1,
         '방공': 2,
         '무기고': 1,
         '획': 1,
         '최순실': 343,
         '허주': 1,
         '적중': 1,
         '청': 24,
         '지구촌': 3,
         '일해': 1,
         '권선': 1,
         '서상일': 1,
         '노동': 8,
         '이슬람': 11,
         '공신': 2,
         '토정비결': 1,
         '표창': 6,
         '자화자찬': 1,
         '음란': 3,
         '대관령': 15,
         '아예': 5,
         '불심': 1,
         '미초아칸': 2,
         '사자성어': 1,
         '명실': 1,
         '제목': 16,
         '정수': 1,
         '김재': 4,
         '편성': 17,
         '유관': 5,
         '꽃지': 2,
      

In [10]:
for min_count in [2,3,5,10]:
    _counter = {word for word ,freq in noun_counter.items() if freq >= min_count}
    print("num of noun (min_count = {}): {:>10}".format(min_count, len(_counter)))

num of noun (min_count = 2):       7772
num of noun (min_count = 3):       5900
num of noun (min_count = 5):       4102
num of noun (min_count = 10):       2424


In [11]:
noun_dict = {word:freq for word, freq in noun_counter.items() if freq >= 5}

def custom_tokenizer(doc):
    return [word for word in twitter.nouns(doc) if word in noun_dict]

print(twitter.nouns(docs[1]))
print(custom_tokenizer(docs[1]))

['서울', '연합뉴스', '이재희', '기자', '소녀시대', '태연', '일', '오후', '서울', '강남구', '코엑스', '어워즈', '페스티벌', '가요대전', '에서', '포즈']
['서울', '연합뉴스', '이재희', '기자', '일', '오후', '서울', '강남구', '코엑스', '어워즈', '페스티벌', '가요대전', '에서', '포즈']


# CountVectorizer(sklearn)

In [12]:
from sklearn.feature_extraction.text import CountVectorizer

In [13]:
vectorizer = CountVectorizer(tokenizer=custom_tokenizer, min_df=0.05, max_df=0.95)
x_sparse = vectorizer.fit_transform(docs)

In [14]:
x_sparse

<1355x165 sparse matrix of type '<class 'numpy.int64'>'
	with 24394 stored elements in Compressed Sparse Row format>

In [15]:
vocab2int = vectorizer.vocabulary_
vocab2int["가운데"]

0

In [16]:
int2vocab = [word for word, index in sorted(vocab2int.items(), key=lambda x:x[1])]

In [17]:
int2vocab[0]

'가운데'

# Sparse matrix  VS  dense matrix

In [18]:
from scipy.io import mmread, mmwrite

In [19]:
mmwrite("./tmp/x.mm", x_sparse)

In [20]:
x_sparse.data.shape

(24394,)

In [21]:
x_dense = x_sparse.todense()

In [22]:
x_dense.shape

(1355, 165)

In [23]:
x_dense

matrix([[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 1, ..., 0, 0, 0],
        [0, 1, 0, ..., 2, 0, 0],
        ..., 
        [0, 0, 1, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0]], dtype=int64)

# vector

In [24]:
import numpy as np

In [25]:
x_tmp = np.random.random((5,10))

In [26]:
x_tmp

array([[ 0.51920146,  0.31762037,  0.96515521,  0.31072992,  0.14253522,
         0.55168291,  0.48213578,  0.5996086 ,  0.97652265,  0.55271547],
       [ 0.37308219,  0.11434363,  0.71412   ,  0.16127547,  0.23278455,
         0.71845842,  0.60344473,  0.8804352 ,  0.54299209,  0.00878586],
       [ 0.3871467 ,  0.49537592,  0.27741711,  0.26833884,  0.22169348,
         0.87439977,  0.84483549,  0.79684809,  0.14485737,  0.936203  ],
       [ 0.53567242,  0.58307243,  0.46666958,  0.87509139,  0.46579071,
         0.29675071,  0.34911564,  0.55757218,  0.94538732,  0.48482294],
       [ 0.12746133,  0.24476583,  0.59129003,  0.66609672,  0.88428533,
         0.8361022 ,  0.43169655,  0.38717502,  0.11062337,  0.45237958]])

In [27]:
# row sum
x_tmp.sum(axis=0)

array([ 1.94256411,  1.75517819,  3.01465193,  2.28153234,  1.94708928,
        3.27739401,  2.7112282 ,  3.2216391 ,  2.7203828 ,  2.43490684])

In [28]:
# column
x_tmp.sum(axis=1)

array([ 5.41790761,  4.34972214,  5.24711578,  5.55994531,  4.73187595])

In [29]:
# word frequency (1doc = row)
x_tmp.sum(axis=0) / x_tmp.sum()

array([ 0.07676127,  0.06935663,  0.11912528,  0.09015574,  0.07694008,
        0.12950765,  0.10713536,  0.12730447,  0.10749711,  0.0962164 ])

In [30]:
x1 = np.random.random((1,10))
x2 = np.random.random((1,10))

score = x1 / (x1 + x2)

In [31]:
score

array([[ 0.30655293,  0.33240456,  0.27458301,  0.18179907,  0.2413788 ,
         0.38623277,  0.70456948,  0.56577241,  0.75124305,  0.5998231 ]])

In [32]:
def keyword_extraction(x_dense, aspect_word_index, int2vocab, threshold=0.7):
    dt_idx = []
    dr_idx = []
    
    # 모든 문서에서 aspect_word의 column 가져오기
    for idx, tf in enumerate(x_dense[:, aspect_word_index]):
        # tf 가 0보다 크면 target (positive) document set, 
        if tf > 0:
            dt_idx.append(idx)
        # tf == 0은 aspect word가 등장하지 않았으므로 reference document set
        else:
            dr_idx.append(idx)
    
    # x_dense에 list로 이뤄진 dt_idx를 넣으면, 해당 index의 rows만을 추려서 submatrix를 만들어줍니다. 
    x_dt = x_dense[dt_idx]
    x_dr = x_dense[dr_idx]
    
    # term proportion matrix를 만듭니다. 
    x_dt = x_dt.sum(axis=0) / x_dt.sum()
    x_dr = x_dr.sum(axis=0) / x_dr.sum()
    
    # scure를 구한 뒤 reshape을 이용하여 column vector 형태로 만들어줍니다. 
    x_score = x_dt / (x_dt + x_dr)
    x_score = np.asarray(x_score).reshape(-1)
    
    # score가 threshold 이상인 단어들의 index를 keywords list에 넣어둡니다. 
    keywords = [(word_idx, score) for word_idx, score in enumerate(x_score)]
    
    # index2word를 이용하여 word index를 키워드로 바꿔줍니다. 
    keywords = [(int2vocab[word_idx], score) for word_idx, score in keywords]
    
    return keywords

In [33]:
for word in ['연합뉴스', '대통령', '최순실']:
    
    if not word in vocab2int:
        continue
        
    print('\n\nAspect word = %s (%d)' % (word, vocab2int[word]))
    
    word_index = vocab2int[word]
    keywords_ = keyword_extraction(x_dense, word_index, int2vocab, threshold=0.7)
    
    # 길이가 1인 단어들 제거
    keywords = [(word, score) for word, score in keywords_ if len(word) > 1]
    
    # 추출된 키워드를 term frequency 기준으로 정렬하여 상위 20개 선택
    keywords_tops = sorted(keywords, key=lambda x:noun_counter.get(x[0], 0), reverse=True)[:20]
    
    for keyword, score in keywords_tops:
        print('%10s\t%.3f' % (keyword, score))



Aspect word = 연합뉴스 (97)
      연합뉴스	1.000
        서울	0.822
        기자	0.914
        오전	0.815
       대통령	0.419
        의원	0.419
        정부	0.790
        대표	0.644
       최순실	0.215
        억원	0.762
        사업	1.000
        의혹	0.223
        수사	0.242
        지역	0.743
        오후	0.416
        내년	0.572
        지난	0.571
        사진	0.303
        국회	0.466
        대한	0.723


Aspect word = 대통령 (35)
      연합뉴스	0.315
        서울	0.324
        기자	0.321
        오전	0.265
       대통령	1.000
        의원	0.565
        정부	0.577
        대표	0.513
       최순실	0.602
        억원	0.113
        사업	0.224
        의혹	0.827
        수사	0.775
        지역	0.251
        오후	0.207
        내년	0.304
        지난	0.492
        사진	0.434
        국회	0.453
        대한	0.584


Aspect word = 최순실 (150)
      연합뉴스	0.344
        서울	0.397
        기자	0.365
        오전	0.524
       대통령	0.795
        의원	0.644
        정부	0.592
        대표	0.168
       최순실	1.000
        억원	0.056
        사업	0.162
        의혹	0.918
        수사	0.934
        지역	0.137
     

In [ ]:
# TODO: Aspect word 없이 문서에서 키워드 추출하는 법